In [ ]:
from bs4 import BeautifulSoup
import urllib2
import codecs
import re

In [5]:
def gettext(string):    
    return re.sub('\s+', ' ', string.strip())


def fun(url):
    flag = -1
    content = urllib2.urlopen(url).read().replace('&nbsp;',' ').replace('&#160;',' ').encode('ascii','ignore')
    soup = BeautifulSoup(content,"lxml")
    alltags = soup.find_all()
    final_str =""
    prev_text =""
    bold_tags = False
    
    
    for tag in alltags:

        try:
            tag_text = gettext(tag.get_text())
            if tag.name=='b' or tag.name =='strong' or 'bold' in tag['style']:

                if '2.01' in tag_text :
                    flag = 0

                    if tag_text != prev_text:
                        #print '0',tag.name,tag_text
                        final_str += tag_text + '\n'
                        prev_text = tag_text
                        

                    #print '1'#,tag_text
                    nexttags = tag.find_all_next()
                    for x in nexttags:
                        x_text = gettext(x.get_text())
                        if 'item' in x_text.lower() :
                            for x_child in x.descendants:
                               # print x_child.name                            
                                if x_child.name == 'b' or tag.name =='strong'or 'bold' in x['style']:
                                    #print '3',x_child.get_text()                                    
                                    return (final_str,flag)

                        #print '2' #,x.name,x_text
                        
                        # if item 2.01 is the last item. Exiting when it sees 'signature'
                        if 'signature' in x_text[:9].lower():
                            print 'si------------------gnature found || bold',url
                            return (final_str,flag)


                        if x_text not in prev_text:                            
                            final_str += x_text +'\n'
                        prev_text = x_text
                    
        except:
            pass
    
    if len(final_str)>0:
        return (final_str,flag)
    else:   
        final_str,flag = notbold(alltags)
        return (final_str,flag)


In [6]:
halcon,flag = fun('https://www.sec.gov/Archives/edgar/data/1494530/000144554618002940/fsd_8k.txt')
print halcon
print flag

attribute error
Item 2.01 Completion of Acquisition or Disposition of Assets. First Trust High Income Long/Short Fund is filing herewith a press release issued today by First Trust Advisors L.P. ("FTA") announcing that the merger of First Trust Strategic High Income Fund II (NYSE: FHY), a closed-end fund managed by FTA, with and into First Trust High Income Long/Short Fund (NYSE: FSD), a closed-end fund managed by FTA, became effective immediately before the opening of the NYSE on June 25, 2018. Item 9.01 Financial Statements and Exhibits. (d) Exhibits Exhibit Number Description 99.1 Press release of First Trust Advisors L.P. dated June 25, 2018. SIGNATURES Pursuant to the requirements of the Securities Exchange Act of 1934, the registrant has duly caused this report to be signed on its behalf by the undersigned hereunto duly authorized. Dated: June 25, 2018 FIRST TRUST HIGH INCOME LONG/SHORT FUND By: /s/ W. Scott Jardine ---------------------------------- Name: W. Scott Jardine Title:

In [488]:
k = re.compile('item 2.01[.]*[ ]*(completion)+|item 2.01[.]*[ ]*(disposition)+')
m = k.search('Item 2.01 disposition adasd'.lower())
print m.span()[0]

0


In [2]:
def notbold(alltags):
    flag = -1
    final_str =""
    prev_text =""
    k = re.compile('item 2.01[.]*[ ]*(completion)+|item 2.01[.]*[ ]*(disposition)+')
    for tag in alltags:
        
        tag_text = gettext(tag.get_text())
        #if tag_text[:9] == 'item 2.01':
        if k.match(tag_text.lower()):
            flag = 1
            if tag_text != prev_text:
                #print '0',tag.name,tag_text
                #print 'prev:',prev_text
                #print 'new:',tag_text
                final_str += tag_text + '\n'
                #print final_str+'\n'+'--------------------'

                prev_text = tag_text

            #print '1'#,tag_text
            try:
                nexttags = tag.next_sibling.find_all_next()
                for x in nexttags:
                    x_text = gettext(x.get_text())
                    #print '33'+'-------\n'+x_text[:10]
                    if 'item' in x_text[:5].lower() :

                        return final_str,flag


                    #print '2' #,x.name,x_text
             # if item 2.01 is the last item. Exiting when it sees 'signature'
                    if 'signature' in x_text[:9].lower():
                        print 'si------------------gnature found || not bold',url
                        return (final_str,flag)

                    if x_text not in final_str:
                        #print '5',x.name,x_text
                        #print 'prev:',prev_text
                        #print 'new:',tag_text                            
                        final_str += x_text +'\n'
                        prev_text = x_text
                        #print final_str+'\n'+'--------------------'
            except AttributeError:
                print 'attribute error'

                    
                
    if len(final_str)>0:
        return (final_str,flag)
    else:
        return (None,flag)

In [18]:
def getText(home_url):
    global item_bold,item_nb,no_item
    content = urllib2.urlopen(home_url).read()
    soup = BeautifulSoup(content)
    

    for link in soup.find_all(id='viewFiling'):  
        
        if link.string[:3] == '8-K':            # Ensures that only 8-K filings, and no attachments are downloaded.
            #print link.string
            pos_arr = []                        # To find the position of the url and the company name
            a = (link.get('href'))
            for i in range(len(a)):
                if a[i] == "'":
                    pos_arr.append(i)
            url = a[pos_arr[0]+1:pos_arr[1]]
            company_name = a[pos_arr[2]+1:pos_arr[3]]
            #print company_name,url
            # Writing the text of the filing to a text file
            ans,flag = fun(url)
            #print 'flag = ',flag
            #print 'length of filing is:' ,len(ans)
            if ans:

                
                if (flag == 0):
                    item_bold += 1
                    #print company_name,url
                    with codecs.open('Data\\Item 2.01 filings_2018_html\\Q1\\'+company_name+"-item_2.01.txt","w","ascii","ignore") as f:
                        f.write(ans)
                if (flag == 1):
                    item_nb += 1
                    print '-------not bold',company_name,url
                    with codecs.open('Data\\Item 2.01 filings_2018_html\\Q1\\'+company_name+"-item_2.01.txt","w","ascii","ignore") as f:
                        f.write(ans)
                    
                #print 'Yes! Item 2.01',company_name,url
                
            else:
                no_item += 1
                print 'No Item 2.01',company_name,url

In [19]:
item_bold = 0
item_nb = 0
no_item = 0
for i in range(1,1402,100):
    if i == 1:
        home_url = "https://searchwww.sec.gov/EDGARFSClient/jsp/EDGAR_MainAccess.jsp?search_text=2.01&sort=Date&formType=Form8K&isAdv=true&stemming=false&numResults=100&fromDate=01/01/2018&toDate=03/31/2018&numResults=100"
        
    else:
        home_url = "https://searchwww.sec.gov/EDGARFSClient/jsp/EDGAR_MainAccess.jsp?search_text=2.01&sort=Date&startDoc="+str(i)+"&numResults=100&isAdv=true&formType=Form8K&fromDate=01/01/2018&toDate=03/31/2018&stemming=false"

    print i
    getText(home_url)
    
    
print item_bold,item_nb,no_item

1
No Item 2.01 POTLATCHDELTIC CORP http://www.sec.gov/Archives/edgar/data/1338749/000156459018007246/pch-8k_20180330.htm
si------------------gnature found || bold http://www.sec.gov/Archives/edgar/data/1354591/000149315218004117/form8-k.htm
101
-------not bold CARLISLE COMPANIES INC http://www.sec.gov/Archives/edgar/data/790051/000079005118000099/cslform8-k03x20x18.htm
201
301
si------------------gnature found || bold http://www.sec.gov/Archives/edgar/data/819926/000121390018002679/f8k_sharingeconomy.htm
No Item 2.01 Amplify Energy Corp http://www.sec.gov/Archives/edgar/data/1521847/000119312518072782/d531735d8k.htm
si------------------gnature found || bold http://www.sec.gov/Archives/edgar/data/1424549/000142454918000001/flapothecadeal3618.htm
401
-------not bold CLEAN HARBORS INC http://www.sec.gov/Archives/edgar/data/822818/000110465918013113/a18-7267_18k.htm
No Item 2.01 General Moly Inc http://www.sec.gov/Archives/edgar/data/1275229/000110465918013549/a18-7324_18k.htm
si----------

In [156]:
print item,no_item

365 31


In [179]:
a = 'Item 2.01              Completion of Acquisition or Disposition of Assets.'
a= re.sub('\s+', ' ', a.lower().strip())
print 'item 2.01 completion' in a

True
